In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("/kaggle/input/deeplearing-project/top_10_tags_encoded.csv")

In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

2025-04-10 05:16:00.861948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744262161.327959      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744262161.476245      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
# Parameters
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 300
EMBEDDING_DIM = 100
EPOCHS = 5
BATCH_SIZE = 128

In [10]:
# Tokenize
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])

In [11]:
# Pad
X_seq = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [12]:
# Labels (one-hot multilabel)
y = df.iloc[:, 1:].values  # assuming clean_text is the first column

In [13]:
# Model
model = Sequential([
    Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
    LSTM(128, return_sequences=False),
    Dense(64, activation='relu'),
    Dense(y.shape[1], activation='sigmoid')  # multilabel output
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1744262359.194596      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744262359.195251      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [14]:
# Train on full data
history = model.fit(
    X_seq, y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)]
)

Epoch 1/5


I0000 00:00:1744262371.630453      94 cuda_dnn.cc:529] Loaded cuDNN version 90300


5514/5514 ━━━━━━━━━━━━━━━━━━━━ 124s 21ms/step - accuracy: 0.6087 - loss: 0.1833
Epoch 2/5
   4/5514 ━━━━━━━━━━━━━━━━━━━━ 1:59 22ms/step - accuracy: 0.8270 - loss: 0.0800

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


5514/5514 ━━━━━━━━━━━━━━━━━━━━ 123s 22ms/step - accuracy: 0.8453 - loss: 0.0739
Epoch 3/5
5514/5514 ━━━━━━━━━━━━━━━━━━━━ 123s 22ms/step - accuracy: 0.8536 - loss: 0.0675
Epoch 4/5
5514/5514 ━━━━━━━━━━━━━━━━━━━━ 123s 22ms/step - accuracy: 0.8592 - loss: 0.0633
Epoch 5/5
5514/5514 ━━━━━━━━━━━━━━━━━━━━ 123s 22ms/step - accuracy: 0.8652 - loss: 0.0598


In [15]:
# Save model
model.save("final_model.h5")